In [2]:
import pathlib

import duckdb
import pandas as pd
from IPython.core.magic import register_cell_magic

# cwd = os.getcwd()
# print(f"CWD: {cwd}")

pd.set_option("display.max_rows", None)  # show all rows
pd.set_option("display.max_columns", None)  # show all columns
pd.set_option("display.max_colwidth", None)  # don't truncate text
pd.set_option("display.expand_frame_repr", False)  # don't wrap across lines


DB_PATH = pathlib.Path(
    "/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/warehouse/plt.duckdb"
)

con = duckdb.connect(database=":memory:")
con.execute(f"ATTACH '{DB_PATH}' AS plt (READ_ONLY)")


# # 1) Where to store the DuckDB database file (change if you want it elsewhere)
# DB_PATH = Path("./warehouse/plt.duckdb")
# DB_PATH.parent.mkdir(parents=True, exist_ok=True)
# con.execute("ATTACH './warehouse/plt.duckdb' AS db")

# con = duckdb.connect(database=":memory:")


@register_cell_magic
def ducksql(line, cell):
    res = con.sql(cell)
    try:
        display(res.df())
    except Exception:
        print("OK")

In [3]:
%%ducksql
-- 🤖 Replace these with your actual absolute paths
-- Example:
-- '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/premier-league.csv'
-- '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/FPL_Transfers_23_24.csv'

-- ✏️ Replace the two absolute paths below
CREATE OR REPLACE TEMP VIEW _pl AS
SELECT DISTINCT trim(club_name) AS club
FROM read_csv_auto('/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/premier-league.csv', header=true)
WHERE club_name IS NOT NULL AND trim(club_name) <> '';

CREATE OR REPLACE TEMP VIEW _fpl_clubs AS
WITH raw AS (
  SELECT DISTINCT trim(club_name) AS club
  FROM read_csv_auto('/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/FPL_Transfers_23_24.csv', header=true)
  WHERE club_name IS NOT NULL AND trim(club_name) <> ''
  UNION
  SELECT DISTINCT
         trim(coalesce(
           regexp_extract(club_involved_name_country, '^(.*?)(?:,| - )', 1),
           club_involved_name_country
         )) AS club
  FROM read_csv_auto('/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/FPL_Transfers_23_24.csv', header=true)
  WHERE club_involved_name_country IS NOT NULL AND trim(club_involved_name_country) <> ''
)
SELECT club
FROM raw
WHERE club IS NOT NULL AND trim(club) <> '';

CREATE OR REPLACE TEMP VIEW _all_clubs AS
SELECT club FROM _pl
UNION
SELECT club FROM _fpl_clubs;

CREATE OR REPLACE TEMP VIEW _countries AS
WITH cr AS (
  SELECT DISTINCT
    trim(coalesce(
      regexp_extract(club_involved_name_country, '^(.*?)(?:,| - )', 1),
      club_involved_name_country
    )) AS club,
    trim(coalesce(
      regexp_extract(club_involved_name_country, ',\\s*([^,]+)$', 1),
      regexp_extract(club_involved_name_country, ' - ([^-]+)$', 1)
    )) AS country
  FROM read_csv_auto('/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/FPL_Transfers_23_24.csv', header=true)
  WHERE club_involved_name_country IS NOT NULL AND trim(club_involved_name_country) <> ''
)
SELECT club, country
FROM cr
WHERE club IS NOT NULL AND trim(club) <> ''
  AND country IS NOT NULL AND trim(country) <> '';

CREATE OR REPLACE TEMP VIEW clubs_map_tmp AS
SELECT club AS raw_key, upper(club) AS cleaned_full_name
FROM _all_clubs
WHERE club IS NOT NULL AND trim(club) <> '';

CREATE OR REPLACE TEMP VIEW club_country_map_tmp AS
SELECT upper(c.club) AS cleaned_full_name, c.country
FROM _countries c
WHERE c.country IS NOT NULL AND trim(c.country) <> '';

OK


In [4]:
%%ducksql

-- 🤖 Replace the two output paths with your repo locations
COPY (SELECT * FROM clubs_map_tmp ORDER BY cleaned_full_name)
TO '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/clubs_map.csv' (HEADER, DELIMITER ',');

COPY (SELECT * FROM club_country_map_tmp ORDER BY cleaned_full_name, country)
TO '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/club_country_map.csv' (HEADER, DELIMITER ',');

OK


In [2]:
%%ducksql

-- Run in DuckDB (same DB your dbt uses)
COPY (
  with raw_names as (
    select upper(trim(club_name))          as raw_key from plt.raw_transfers
    union
    select upper(trim(club_involved_name)) as raw_key from plt.raw_transfers
  ),
  dedup as (
    select distinct raw_key from raw_names where raw_key <> ''
  )
  select
    raw_key,
    raw_key as cleaned_full_name,
    null    as abbreviation,
    'Unknown' as country
  from dedup
  order by 1
) TO '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/clubs_map.csv'
  (HEADER, DELIMITER ',');

OK


In [3]:
%%ducksql

COPY (
  with all_names as (
    select upper(trim(club_name))          as nm, 1 as is_pl_side from plt.raw_transfers
    union all
    select upper(trim(club_involved_name)) as nm, 0 as is_pl_side from plt.raw_transfers
  ),
  dedup as (
    select nm,
           max(is_pl_side) as ever_pl_side
    from all_names
    where nm <> ''
    group by 1
  )
  select
    nm as cleaned_full_name,
    case
      when nm in ('CARDIFF CITY','SWANSEA CITY','NEWPORT COUNTY','WREXHAM') then 'Wales'
      when ever_pl_side = 1 then 'England'
      else 'Unknown'
    end as country
  from dedup
  order by 1
) TO '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/club_country_map.csv'
  (HEADER, DELIMITER ',');

OK


In [5]:
%%ducksql

-- Option A (DuckDB / Jupyter)
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema='plt' AND table_name='raw_transfers'
ORDER BY ordinal_position;

-- or just preview a sample
SELECT * FROM plt.raw_transfers LIMIT 5;

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Middlesbrough FC,Tommy Wright,26,Left Winger,Leicester,€910Th.,in,Summer,0.91,Premier League,1992,1992/1993
1,Middlesbrough FC,Jonathan Gittens,28,defence,Southampton,€250Th.,in,Summer,0.25,Premier League,1992,1992/1993
2,Middlesbrough FC,Chris Morris,28,Right-Back,Celtic,?,in,Summer,NA,Premier League,1992,1992/1993
3,Middlesbrough FC,Ben Roberts,17,Goalkeeper,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993
4,Middlesbrough FC,Andy Todd,17,Centre-Back,Boro U18,-,in,Summer,NA,Premier League,1992,1992/1993


In [7]:
%%ducksql

-- If it's already attached read-only, detach first
DETACH DATABASE IF EXISTS plt;

-- Re-attach the SAME file, writable (default)
ATTACH '/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/warehouse/plt.duckdb' AS plt;

-- sanity check
PRAGMA database_list;

,seq,name,file
0,1146,memory,None
1,2485,plt,/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/warehouse/plt.duckdb


In [8]:
%%ducksql
SELECT table_name, table_type
FROM information_schema.tables
WHERE table_schema='main' AND table_name IN ('raw_pl','raw_transfers');

,table_name,table_type
0,raw_pl,BASE TABLE


In [9]:
%%ducksql
DROP VIEW IF EXISTS plt.main.raw_transfers;
DROP TABLE IF EXISTS plt.main.raw_transfers;

OK
